# PyMC3 example

PyMC3 implimentation provided by Junpeng Lao, see https://discourse.pymc.io/t/bayes-nets-belief-networks-and-pymc/5150/2?u=drbenvincent


In [1]:
import pymc3 as pm
import numpy as np

import theano
import theano.tensor as tt

In [2]:
lookup_table = theano.shared(np.asarray([
    [[.99, .01], [.1, .9]],
    [[.9, .1], [.1, .9]]]))

def f(smoker, covid):
    return lookup_table[smoker, covid]

with pm.Model() as m:
    smoker = pm.Categorical('smoker', [.75, .25])
    covid = pm.Categorical('covid', [.9, .1])
    hospital = pm.Categorical('hospital', f(smoker, covid))
    prior_trace = pm.sample_prior_predictive(100000)

In [3]:
predict_proba0 = prior_trace['covid'][
    (prior_trace['smoker'] == 0)
  & (prior_trace['hospital'] == 1)].mean()

predict_proba1 = prior_trace['covid'][
    (prior_trace['smoker'] == 1)
  & (prior_trace['hospital'] == 1)].mean()

print(f'P(covid|¬smoking, hospital) is {predict_proba0}')
print(f'P(covid|smoking, hospital) is {predict_proba1}')

P(covid|¬smoking, hospital) is 0.9036904925229798
P(covid|smoking, hospital) is 0.494931687968268


In [4]:
lookup_table[0,0].eval()

array([0.99, 0.01])

In [5]:
lookup_table[0,1].eval()

array([0.1, 0.9])

In [6]:
lookup_table[1,0].eval()

array([0.9, 0.1])

In [7]:
lookup_table[1,1].eval()

array([0.1, 0.9])